<a href="https://colab.research.google.com/github/Amanda9805/Detecting-Machine-Generated-Texts/blob/train-model/Afriberta_model2Combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers adapters datasets fsspec evaluate shap nltk lime textstat

In [ ]:
import os
import re
import json
import gc
import warnings
import torch
import nltk
import evaluate
import shap
import lime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from nltk.tokenize import word_tokenize
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from peft import LoraConfig, get_peft_model
from textstat import flesch_reading_ease, automated_readability_index
from huggingface_hub import login
from datasets import load_dataset, Dataset, ClassLabel, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    pipeline
)

# Initial setup
nltk.download('punkt_tab')

torch.manual_seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')

In [ ]:
# Basic text cleaning
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\\S+|www\\S+|https\\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Load machine-generated text from a JSONL file
def load_machine_text(file_path):
    data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.strip():
                    data.append(json.loads(line))
        df = pd.DataFrame(data)
        # Handle cases where text is a list of strings
        if not df.empty and isinstance(df['text'].iloc[0], list):
            df['text'] = df['text'].apply(lambda x: ' '.join(x))
        return df
    except FileNotFoundError:
        print(f"Error: Machine-generated text file not found at {file_path}")
        return pd.DataFrame()

# Load, process and balance data for a given language
def prepare_dataset(lang_code, human_data_source, machine_data_path, output_csv_path):
    print(f"Preparing dataset for language: {lang_code.upper()}")

    # Load Human Data (Label = 0)
    human_dataset = load_dataset(human_data_source, lang_code)

    human_df = human_dataset['train'].to_pandas()
    human_df = human_df[['text']].dropna()
    human_df['label'] = 0
    human_df['language'] = lang_code
    print(f"Loaded {len(human_df)} human-written data.")

    # Load Machine Data (Label = 1)
    machine_df = load_machine_text(machine_data_path)
    if machine_df.empty:
        return None
    machine_df = machine_df[['text']].dropna()
    machine_df['label'] = 1
    machine_df['language'] = lang_code
    print(f"Loaded {len(machine_df)} machine-generated data.")

    # Combine and Clean
    combined_df = pd.concat([human_df, machine_df], ignore_index=True)
    combined_df['text'] = combined_df['text'].apply(clean_text)
    combined_df.dropna(subset=['text'], inplace=True)
    # combined_df = combined_df[combined_df['text'].str.len() > 10] # Optional: Remove very short texts

    # Create Balanced Dataset
    min_class_size = combined_df['label'].value_counts().min()
    balanced_df = combined_df.groupby('label').sample(n=min_class_size, random_state=42)
    balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
    print(f"Created a balanced dataset with {len(balanced_df)} total samples ({min_class_size} per class).")

    # Save and return dataset
    balanced_df.to_csv(output_csv_path, index=False, encoding='utf-8')
    print(f"Balanced dataset saved to {output_csv_path}")

    return balanced_df

In [ ]:
# Create dummy data files if they don't exist, as they are loaded in the notebook
if not os.path.exists('zulu_mg_text.jsonl'):
    with open('zulu_mg_text.jsonl', 'w') as f:
        f.write('{"text": "Lokhu umbhalo owenziwe ngomshini mayelana namasiko akwaZulu."}\n')
        f.write('{"text": "UNogwaja noFudu babengabangani abakhulu ehlathini."}\n')

if not os.path.exists('eng_mg_data.jsonl'):
    with open('eng_mg_data.jsonl', 'w') as f:
        f.write('{"text": "This is machine-generated text about South African culture."}\n')
        f.write('{"text": "The quick brown fox jumps over the lazy dog in Johannesburg."}\n')

# Prepare isiZulu Dataset
zulu_dataset = prepare_dataset(
    lang_code='zul',
    human_data_source='dsfsi/vukuzenzele-monolingual',
    machine_data_path='zulu_mg_text.jsonl',
    output_csv_path='balanced_zulu_texts.csv'
)

# Prepare English Dataset
english_dataset = prepare_dataset(
    lang_code='eng',
    human_data_source='dsfsi/vukuzenzele-monolingual',
    machine_data_path='eng_mg_data.jsonl',
    output_csv_path='balanced_english_texts.csv'
)

print("\nZulu Dataset Sample:")
print(pd.DataFrame(zulu_dataset).head())
print("\nEnglish Dataset Sample:")
print(pd.DataFrame(english_dataset).head())

In [ ]:
class ZuluTextClassifier:
    def __init__(self, model_name='castorini/afriberta_base', max_length=512):
        print(f"Loading model: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        self.max_length = max_length
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        print(f"Model initialized on {self.device}")

    def prepare_data(self, df):
        X = df['text'].tolist()
        y = LabelEncoder().fit_transform(df['label'])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        return X_train, X_test, y_train, y_test

    def create_datasets(self, X_train, X_test, y_train, y_test):
        train_encodings = self.tokenizer(X_train, truncation=True, padding=True, max_length=self.max_length)
        test_encodings = self.tokenizer(X_test, truncation=True, padding=True, max_length=self.max_length)

        train_data = {
            'input_ids': train_encodings['input_ids'],
            'attention_mask': train_encodings['attention_mask'],
            'labels': y_train
        }
        test_data = {
            'input_ids': test_encodings['input_ids'],
            'attention_mask': test_encodings['attention_mask'],
            'labels': y_test
        }

        train_dataset = Dataset.from_dict(train_data)
        test_dataset = Dataset.from_dict(test_data)
        return train_dataset, test_dataset

    def setup_lora(self, use_lora=True):
        if use_lora:
            print("Setting up LoRA configuration...")
            lora_config = LoraConfig(
                r=16,
                lora_alpha=32,
                target_modules=["query", "key", "value"],
                lora_dropout=0.1,
                bias="none",
                task_type="SEQ_CLS"
            )
            self.model = get_peft_model(self.model, lora_config)
            print(f"LoRA enabled. Trainable parameters: {self.model.num_parameters()}")

    def train(self, train_dataset, eval_dataset):
        self.setup_lora(use_lora=True)
        training_args = TrainingArguments(
            output_dir='./classifier_results',
            num_train_epochs=5,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            warmup_steps=100,
            weight_decay=0.01,
            learning_rate=5e-5,
            logging_steps=10,
            eval_strategy="steps",
            eval_steps=50,
            save_strategy="steps",
            save_steps=50,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            fp16=torch.cuda.is_available(),
            report_to="none"
        )
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer
        )
        print("Starting training...")
        trainer.train()
        trainer.save_model()
        self.tokenizer.save_pretrained('./classifier_results')
        return trainer

    def evaluate(self, test_dataset, trainer=None):
        if trainer is None:
            trainer = Trainer(model=self.model)

        predictions = trainer.predict(test_dataset)
        y_pred = np.argmax(predictions.predictions, axis=1)
        y_true = predictions.label_ids

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
        precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
            y_true, y_pred, average=None
        )

        results = {
            'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1,
            'precision_per_class': precision_per_class, 'recall_per_class': recall_per_class,
            'f1_per_class': f1_per_class, 'support': support,
            'y_true': y_true, 'y_pred': y_pred
        }
        return results

    def print_evaluation_results(self, results):
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)
        print(f"Overall Accuracy: {results['accuracy']:.4f}")
        print(f"Weighted Precision: {results['precision']:.4f}")
        print(f"Weighted Recall: {results['recall']:.4f}")
        print(f"Weighted F1-Score: {results['f1']:.4f}")
        print("\nPer-Class Results:")

        class_names = ['Human', 'Machine']

        for i in range(len(class_names)):
            print(f"{class_names[i]}:")
            print(f"  Precision: {results['precision_per_class'][i]:.4f}")
            print(f"  Recall: {results['recall_per_class'][i]:.4f}")
            print(f"  F1-Score: {results['f1_per_class'][i]:.4f}")
            print(f"  Support: {results['support'][i]}")

        print("\nClassification Report:")
        print(classification_report(results['y_true'], results['y_pred'], target_names=class_names))

    def predict_single_text(self, text):
        self.model.eval()
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        encoding = {k: v.to(self.device) for k, v in encoding.items()}

        with torch.no_grad():
            outputs = self.model(**encoding)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=-1).item()
            confidence = predictions.max().item()

        class_names = ['Human', 'Machine']

        return {
            'predicted_class': class_names[predicted_class], 'confidence': confidence,
            'probabilities': {'Human': predictions[0][0].item(), 'Machine': predictions[0][1].item()}
        }

In [ ]:
class TextExplainabilityAnalyzer:
    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.class_names = ['Human', 'Machine']
        self.lime_explainer = LimeTextExplainer(class_names=self.class_names)

    def predict_proba_for_lime(self, texts):
        predictions = []
        self.model.eval()
        for text in texts:
            encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
            encoding = {k: v.to(self.device) for k, v in encoding.items()}
            with torch.no_grad():
                outputs = self.model(**encoding)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predictions.append(probs.cpu().numpy()[0])
        return np.array(predictions)

    def explain_with_lime(self, text, num_features=10):
        explanation = self.lime_explainer.explain_instance(
            text, self.predict_proba_for_lime, num_features=num_features, num_samples=500
        )
        return explanation

    def explain_with_shap(self, texts, background_texts=None):
        def model_wrapper(texts):
            return self.predict_proba_for_lime(texts)
        if background_texts is None:
            background_texts = texts[:20]
        explainer = shap.Explainer(model_wrapper, background_texts)
        shap_values = explainer(texts)
        return shap_values

    def analyze_feature_importance(self, test_texts, test_labels, sample_ratio=0.3):
        results = {'human_features': [], 'machine_features': [], 'common_patterns': {}}
        total_samples = len(test_texts)
        num_samples = int(total_samples * sample_ratio)
        print(f"Analyzing {num_samples} samples ({sample_ratio*100:.0f}%) out of {total_samples} total samples")
        sample_indices, _ = train_test_split(range(len(test_texts)), test_size=1-sample_ratio, stratify=test_labels, random_state=42)
        print(f"Selected {len(sample_indices)} samples for analysis")
        for i, idx in enumerate(tqdm(sample_indices, desc="Generating LIME explanations")):
            text, true_label = test_texts[idx], test_labels[idx]
            if not text or text.isspace():
                continue
            try:
                explanation = self.explain_with_lime(text)
                features = explanation.as_list()
                if true_label == 0:
                    results['human_features'].extend([f[0] for f in features if f[1] > 0])
                else:
                    results['machine_features'].extend([f[0] for f in features if f[1] > 0])
            except Exception as e:
                print(f"Error generating LIME explanation for text at index {idx}: {e}")
        results['human_patterns'] = Counter(results['human_features']).most_common(20)
        results['machine_patterns'] = Counter(results['machine_features']).most_common(20)
        return results

    def linguistic_feature_analysis(self, texts, labels, sample_ratio=0.3):
        total_samples = len(texts)
        num_samples = int(total_samples * sample_ratio)
        sample_indices, _ = train_test_split(range(len(texts)), test_size=1-sample_ratio, stratify=labels, random_state=42)
        sampled_texts, sampled_labels = [texts[i] for i in sample_indices], [labels[i] for i in sample_indices]

        print(f"Running linguistic analysis on {len(sampled_texts)} samples ({sample_ratio*100:.0f}%)")

        features = {'avg_sentence_length': [], 'lexical_diversity': [], 'repetition_score': []}

        for text in tqdm(sampled_texts, desc="Computing linguistic features"):
            sentences = re.split(r'[.!?]+', text)
            avg_sent_len = np.mean([len(s.split()) for s in sentences if s.strip()])
            features['avg_sentence_length'].append(avg_sent_len)
            words = text.lower().split()
            lexical_div = len(set(words)) / len(words) if words else 0
            features['lexical_diversity'].append(lexical_div)
            word_counts = Counter(words)
            repetition = sum(count - 1 for count in word_counts.values()) / len(words) if words else 0
            features['repetition_score'].append(repetition)

        human_features = {k: [v[i] for i, label in enumerate(sampled_labels) if label == 0] for k, v in features.items()}
        machine_features = {k: [v[i] for i, label in enumerate(sampled_labels) if label == 1] for k, v in features.items()}

        return human_features, machine_features, features

    def plot_feature_distributions(self, human_features, machine_features):
        fig, axes = plt.subplots(1, 3, figsize=(15, 5)) # Adjusted for 3 features
        axes = axes.ravel()
        feature_names = list(human_features.keys())

        for i, feature in enumerate(feature_names):
            ax = axes[i]
            ax.hist(human_features[feature], alpha=0.7, label='Human', bins=20)
            ax.hist(machine_features[feature], alpha=0.7, label='Machine', bins=20)
            ax.set_title(feature.replace('_', ' ').title())
            ax.set_xlabel('Value'); ax.set_ylabel('Frequency'); ax.legend()
        plt.tight_layout()

        return fig

    def error_analysis(self, test_texts, test_labels, predictions, sample_ratio=0.3):
        all_errors = [{'text': test_texts[i], 'true_label': self.class_names[true_label], 'predicted_label': self.class_names[pred_label], 'index': i} for i, (true_label, pred_label) in enumerate(zip(test_labels, predictions)) if true_label != pred_label]
        num_errors_to_analyze = max(1, int(len(all_errors) * sample_ratio))
        errors_to_analyze = np.random.choice(all_errors, min(num_errors_to_analyze, len(all_errors)), replace=False) if all_errors else []
        print(f"Found {len(all_errors)} total errors, analyzing {len(errors_to_analyze)} in detail")
        error_analysis = {
            'total_errors': len(all_errors), 'false_positives': len([e for e in all_errors if e['true_label'] == 'Human']),
            'false_negatives': len([e for e in all_errors if e['true_label'] == 'Machine']), 'examples': all_errors[:10]
        }

        error_explanations = []
        for error in tqdm(errors_to_analyze[:5], desc="Generating error explanations"):
            try:
                explanation = self.explain_with_lime(error['text'])
                error_explanations.append({'error': error, 'explanation': explanation.as_list()})
            except Exception as e:
                print(f"Error generating explanation: {e}")

        error_analysis['explanations'] = error_explanations
        return error_analysis

    def generate_explanation_report(self, test_texts, test_labels, predictions, sample_ratio=0.3):
        print("="*60 + "\nMODEL EXPLAINABILITY ANALYSIS REPORT\n" + f"Analyzing {sample_ratio*100:.0f}% of data for efficiency\n" + "="*60)
        print("\n1. FEATURE IMPORTANCE ANALYSIS\n" + "-" * 40)

        feature_analysis = self.analyze_feature_importance(test_texts, test_labels, sample_ratio)

        print("Top features indicating HUMAN text:"); [print(f"  - '{f}': {c} occurrences") for f, c in feature_analysis['human_patterns'][:10]]
        print("\nTop features indicating MACHINE text:"); [print(f"  - '{f}': {c} occurrences") for f, c in feature_analysis['machine_patterns'][:10]]
        print("\n2. LINGUISTIC FEATURE ANALYSIS\n" + "-" * 40)

        human_features, machine_features, _ = self.linguistic_feature_analysis(test_texts, test_labels, sample_ratio)
        for name in human_features.keys():
            print(f"{name.replace('_', ' ').title()}:\n  Human avg: {np.mean(human_features[name]):.3f}, Machine avg: {np.mean(machine_features[name]):.3f}")

        print("\n3. ERROR ANALYSIS\n" + "-" * 40)

        error_analysis_res = self.error_analysis(test_texts, test_labels, predictions, sample_ratio)

        print(f"Total classification errors: {error_analysis_res['total_errors']}\nFalse positives (Human → Machine): {error_analysis_res['false_positives']}\nFalse negatives (Machine → Human): {error_analysis_res['false_negatives']}")
        print("\nExample misclassifications:")

        for i, ex in enumerate(error_analysis_res['examples'][:3]):
            print(f"  {i+1}. True: {ex['true_label']}, Predicted: {ex['predicted_label']}\n     Text: {ex['text'][:100]}...")

        return {'feature_analysis': feature_analysis, 'linguistic_analysis': (human_features, machine_features), 'error_analysis': error_analysis_res}

def add_explainability_to_classifier(classifier, test_texts, test_labels, predictions, sample_ratio=0.3):
    explainer = TextExplainabilityAnalyzer(classifier.model, classifier.tokenizer, classifier.device)
    report = explainer.generate_explanation_report(test_texts, test_labels, predictions, sample_ratio)

    human_features, machine_features = report['linguistic_analysis']

    fig = explainer.plot_feature_distributions(human_features, machine_features)

    plt.show()

    return explainer, report

In [ ]:
# general-purpose class
class TextClassifier(ZuluTextClassifier):
    pass

def run_experiment(model_name, dataset_df, language_name, run_explainability=False):
    """Runs a complete fine-tuning and evaluation experiment for our datasets."""

    print("\n" + "="*80 + f"\nSTARTING EXPERIMENT: Model='{model_name}', Language='{language_name}'\n" + "="*80)

    classifier = TextClassifier(model_name=model_name)
    X_train, X_test, y_train, y_test = classifier.prepare_data(dataset_df)

    train_dataset, test_dataset = classifier.create_datasets(X_train, X_test, y_train, y_test)
    trainer = classifier.train(train_dataset, test_dataset)

    results = classifier.evaluate(test_dataset, trainer)
    classifier.print_evaluation_results(results)

    if run_explainability:
        print("\n--- Running Explainability Analysis ---")
        add_explainability_to_classifier(classifier, X_test, y_test, results['y_pred'], sample_ratio=0.2)

    del classifier, trainer, train_dataset, test_dataset

    torch.cuda.empty_cache()
    gc.collect()

    return {"model_name": model_name, "language": language_name, "accuracy": results['accuracy'], "f1_score": results['f1'], "precision": results['precision'], "recall": results['recall']}

def run_zero_shot_experiment(model_name, source_df, target_df):
    """
    Trains a model on a source language and evaluates it on a target language (zero-shot).
    """
    print("\n" + "="*80)
    print(f"STARTING ZERO-SHOT EXPERIMENT: Model='{model_name}', Source='English', Target='isiZulu'")
    print("="*80)

    # Initialize classifier
    classifier = TextClassifier(model_name=model_name)

    # Prepare source data for training
    X_train_src, X_test_src, y_train_src, y_test_src = classifier.prepare_data(source_df)
    train_dataset_src, eval_dataset_src = classifier.create_datasets(X_train_src, X_test_src, y_train_src, y_test_src)

    # Train on source language (English)
    print("\n--- Training on source language (English) ---")
    trainer = classifier.train(train_dataset_src, eval_dataset_src)

    # Prepare target data for evaluation
    print("\n--- Evaluating on target language (isiZulu) ---")
    _ , X_test_tgt, _, y_test_tgt = classifier.prepare_data(target_df)

    # Directly create the target test dataset
    target_encodings = classifier.tokenizer(X_test_tgt, truncation=True, padding=True, max_length=classifier.max_length)
    target_test_data = {
        'input_ids': target_encodings['input_ids'],
        'attention_mask': target_encodings['attention_mask'],
        'labels': y_test_tgt
    }
    test_dataset_tgt = Dataset.from_dict(target_test_data)

    # Evaluate on target language (Zulu)
    results = classifier.evaluate(test_dataset_tgt, trainer)
    classifier.print_evaluation_results(results)

    # Clean up
    del classifier, trainer
    torch.cuda.empty_cache()
    gc.collect()

    return {
        "model_name": f"{model_name} (Zero-Shot)",
        "language": "zul",
        "accuracy": results['accuracy'],
        "f1_score": results['f1'],
        "precision": results['precision'],
        "recall": results['recall'],
    }

def full_project_pipeline():
    """Main function to run the entire pipeline as described in the proposal."""

    MODELS_TO_TEST = ['castorini/afriberta_base', 'xlm-roberta-base']
    DATASETS = {"zul": zulu_dataset, "eng": english_dataset}
    all_results = []

    for lang_code, df in DATASETS.items():
        for model in MODELS_TO_TEST:
            is_primary_exp = (lang_code == 'zul' and model == 'castorini/afriberta_base')
            result = run_experiment(model, df, lang_code, run_explainability=is_primary_exp)
            all_results.append(result)

    for model in MODELS_TO_TEST:
        zero_shot_result = run_zero_shot_experiment(model, DATASETS['eng'], DATASETS['zul'])
        all_results.append(zero_shot_result)

    print("\n\n" + "#"*80 + "\n" + " " * 20 + "FINAL COMPARATIVE ANALYSIS REPORT\n" + "#"*80)
    results_df = pd.DataFrame(all_results)
    print("\n--- Performance Metrics Across All Experiments ---")
    print(results_df)

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 7))
    pivot_df = results_df.pivot(index='language', columns='model_name', values='f1_score')
    pivot_df.plot(kind='bar', ax=ax, width=0.8)
    ax.set_title('Comparative F1-Scores: Model Performance on English vs. isiZulu', fontsize=16)
    ax.set_ylabel('F1-Score', fontsize=12); ax.set_xlabel('Language', fontsize=12)
    ax.tick_params(axis='x', rotation=0); ax.legend(title='Model & Training Strategy')
    plt.tight_layout(); plt.show()

if 'zulu_dataset' in locals() and 'english_dataset' in locals():
  full_project_pipeline()
else:
  print("Data preparation failed or was skipped. Cannot run the main pipeline.")

In [ ]:
if __name__ == "__main__":
    # The interactive CLI part can be run after the main pipeline finishes
    print("\nStarting interactive session...")

    try:
        # Load the last saved model and tokenizer for interaction
        final_classifier = TextClassifier()
        final_classifier.model = AutoModelForSequenceClassification.from_pretrained('./classifier_results')
        final_classifier.tokenizer = AutoTokenizer.from_pretrained('./classifier_results')
        final_classifier.model.to(final_classifier.device)

        print("Zulu Text Classification Tool")
        print("Enter Zulu text to classify. Type 'exit' to quit.")
        while True:
            user_input = input("\nEnter Zulu text: ").strip()
            if user_input.lower() == "exit":
                print("Exiting...")
                break
            if not user_input:
                print("Please enter some text.")
                continue
            prediction = final_classifier.predict_single_text(user_input)
            print(f"\nPrediction for '{user_input}':")
            print(f"Predicted: {prediction['predicted_class']} (Confidence: {prediction['confidence']:.4f})")
            print(f"Probabilities: Human - {prediction['probabilities']['Human']:.4f}, Machine - {prediction['probabilities']['Machine']:.4f}")
            print("-" * 50)
    except Exception as e:
        print(f"\nCould not start interactive session. No model found or an error occurred: {e}")